In [1]:
import torchio as tio
import os
import nibabel as nib
import numpy as np
import torch
import pandas as pd

from monai.metrics import compute_average_surface_distance, get_confusion_matrix
from collections.abc import Sequence
from monai.metrics.utils import get_mask_edges, get_surface_distance, ignore_background, prepare_spacing
from monai.utils import convert_data_type, convert_to_tensor
from tqdm import tqdm

from scipy import ndimage
#from scipy.ndimage import label

In the future `np.bool` will be defined as the corresponding NumPy scalar.
In the future `np.bool` will be defined as the corresponding NumPy scalar.
In the future `np.bool` will be defined as the corresponding NumPy scalar.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin C:\Users\user\anaconda3\envs\ailab\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116.dll
CUDA SETUP: CUDA runtime path found: C:\Users\user\anaconda3\envs\ailab\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary C:\Users\user\anaconda3\envs\ailab\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116.dll...


In [2]:
# metric class
class Metric:
    def __init__(self, GT, PRED):
        self.gt = GT
        self.pred = PRED
        
        # metric list
        self.DC_list = []
        self.DG_numerator = []
        self.DG_denominator = []
        self.Jaccard_list = []
        self.VOE_list = []
        self.RVD_list = []
        self.RAVD_list = []
        self.ASSD_list = []
        self.RMSD_list = []
        self.MSSD_list = []
        self.TP_list = []
        self.TN_list = []
        self.FP_list = []
        self.FN_list = []
        self.SEN_list = []
        self.SPE_list = []
        
    def cal_DC_and_DG(self):
        # inersection, union, dice 계산
        intersection = (self.pred*self.gt).sum()
        union = self.pred.sum() + self.gt.sum()
        dice = 2*intersection/union
        
        self.DG_numerator.append(intersection)
        self.DG_denominator.append(union)
        self.DC_list.append(dice) # save
    
    def cal_Jaccard_and_VOE(self):
        intersection = (self.pred*self.gt).sum()
        union = self.pred.sum() + self.gt.sum() - (self.pred*self.gt).sum()
        jaccard = intersection/union
        
        self.Jaccard_list.append(jaccard)
        self.VOE_list.append(1-jaccard)
    
    # RAVD equation: by CotepRes-Net
    def cal_RVD_and_RAVD(self):
        numerator = self.pred.sum() - self.gt.sum()
        denominator = self.pred.sum() # another paper mentioned, self.gt.sum()
        RVD = numerator/denominator
        
        self.RVD_list.append(RVD)
        self.RAVD_list.append(abs(RVD))
    
    # average symmetric surface distance (ASD or ASSD)
    def cal_ASSD(self):
        # monai module
        # compute_average_surface_distance
        assd = compute_average_surface_distance(self.pred, self.gt, symmetric=True)
        
        self.ASSD_list.append(assd)
    
    # root mean symmetric surface distance (RMSD)
    def cal_RMSD(self):
        rmsd = self.compute_root_mean_surface_distance(symmetric=True)
        
        self.RMSD_list.append(rmsd)
        
    # Maximum symmetric surface distance (MSD)
    def cal_MSSD(self):
        mssd = self.compute_maximum_surface_distance()
        
        self.MSSD_list.append(mssd)
    
    def cal_CONFUSION_MATRIX(self):
        pred, gt = torch.Tensor(self.pred), torch.Tensor(self.gt)
        confusion_matrix = get_confusion_matrix(pred, gt, include_background=False)
        tp, fp, tn, fn = confusion_matrix[0,0,...] #  3D -> 1D
        SEN = tp/(tp+fn)
        SPE = tn/(tn+fp)
        
        self.TP_list.append(tp)
        self.TN_list.append(tn)
        self.FP_list.append(fp)
        self.FN_list.append(fn)
        
        self.SEN_list.append(SEN)
        self.SPE_list.append(SPE)
    
    def print_metric(self):
        print("DC score(upper):", self.DC_list[0])
        #print("DG score(upper):", self.DG_numerator[0]/self.DG_denominator[0])
        print("Jaccard score(upper):", self.Jaccard_list[0])
        print("VOE score(lower):", self.VOE_list[0])
        #print("RVD score(lower):", self.RVD_list[0])
        print("RAVD score(lower):", self.RAVD_list[0])
        print("ASSD score(lower):", self.ASSD_list[0].item())
        print("RMSD score(lower):", self.RMSD_list[0].item())
        print("MSSD score(lower):", self.MSSD_list[0].item())
        print("SEN score(upper):", self.SEN_list[0].item())
        print("SPE score(upper):", self.SPE_list[0].item())
    
    def save_metric(self):
        return self.DC_list[0], self.Jaccard_list[0], self.VOE_list[0], self.RAVD_list[0], self.ASSD_list[0].item(), \
                    self.RMSD_list[0].item(), self.MSSD_list[0].item(), self.SEN_list[0].item(), self.SPE_list[0].item()
    
    def all_calculation(self):
        print("######### Start! ######### ")
        self.cal_DC_and_DG()
        self.cal_Jaccard_and_VOE()
        self.cal_RVD_and_RAVD()
        self.cal_ASSD()
        self.cal_RMSD()
        self.cal_MSSD()
        self.cal_CONFUSION_MATRIX()
        print("######### Done! ######### ")
    
    ############### Refereces by Monai
    def compute_root_mean_surface_distance(
        self,
        include_background = False,
        symmetric = True,
        distance_metric = "euclidean",
        spacing = None):

        if not include_background:
            y_pred, y = ignore_background(y_pred=self.pred, y=self.gt)
        else:
            y_pred, y = self.pred.copy(), self.gt.copy()

        y_pred = convert_data_type(y_pred, output_type=torch.Tensor, dtype=torch.float)[0]
        y = convert_data_type(y, output_type=torch.Tensor, dtype=torch.float)[0]

        if y.shape != y_pred.shape:
            raise ValueError(f"y_pred and y should have same shapes, got {y_pred.shape} and {y.shape}.")

        batch_size, n_class = y_pred.shape[:2]
        asd = torch.empty((batch_size, n_class), dtype=torch.float32, device=y_pred.device)

        img_dim = y_pred.ndim - 2
        spacing_list = prepare_spacing(spacing=spacing, batch_size=batch_size, img_dim=img_dim)

        for b, c in np.ndindex(batch_size, n_class):
            _, distances, _ = self.get_edge_surface_distance(
                y_pred[b, c],
                y[b, c],
                distance_metric=distance_metric,
                spacing=spacing_list[b],
                symmetric=symmetric,
                class_index=c,
            )
            surface_distance = torch.cat(distances)
            asd[b, c] = torch.tensor(np.nan) if surface_distance.shape == (0,) else torch.sqrt(torch.mean((surface_distance)**2))
            
        return convert_data_type(asd, output_type=torch.Tensor, device=y_pred.device, dtype=torch.float)[0]
    
    # Make MSSD
    def compute_maximum_surface_distance(
        self,
        include_background = False,
        symmetric = True,
        distance_metric = "euclidean",
        spacing = None):

        if not include_background:
            y_pred, y = ignore_background(y_pred=self.pred, y=self.gt)
        else:
            y_pred, y = self.pred.copy(), self.gt.copy()

        y_pred = convert_data_type(y_pred, output_type=torch.Tensor, dtype=torch.float)[0]
        y = convert_data_type(y, output_type=torch.Tensor, dtype=torch.float)[0]

        if y.shape != y_pred.shape:
            raise ValueError(f"y_pred and y should have same shapes, got {y_pred.shape} and {y.shape}.")

        batch_size, n_class = y_pred.shape[:2]
        asd = torch.empty((batch_size, n_class), dtype=torch.float32, device=y_pred.device)

        img_dim = y_pred.ndim - 2
        spacing_list = prepare_spacing(spacing=spacing, batch_size=batch_size, img_dim=img_dim)

        for b, c in np.ndindex(batch_size, n_class):
            _, distances, _ = self.get_edge_surface_distance(
                y_pred[b, c],
                y[b, c],
                distance_metric=distance_metric,
                spacing=spacing_list[b],
                symmetric=symmetric,
                class_index=c,
            )
            surface_distance = torch.cat(distances)
            asd[b, c] = torch.tensor(np.nan) if surface_distance.shape == (0,) else max(surface_distance)
            
        return convert_data_type(asd, output_type=torch.Tensor, device=y_pred.device, dtype=torch.float)[0]
    
    # get_edge_surface_distance
    def get_edge_surface_distance(
        self,
        y_pred,
        y,
        distance_metric = "euclidean",
        spacing: "int | float | np.ndarray | Sequence[int | float] | None" = None,
        use_subvoxels: bool = False,
        symmetric: bool = False,
        class_index: int = -1):
        
        edges_spacing = None
        if use_subvoxels:
            edges_spacing = spacing if spacing is not None else ([1] * len(y_pred.shape))
        
        (edges_pred, edges_gt, *areas) = get_mask_edges(y_pred, y, crop=True, spacing=edges_spacing, always_return_as_numpy=False)
        
        if symmetric:
            distances = (
                get_surface_distance(edges_pred, edges_gt, distance_metric, spacing),
                get_surface_distance(edges_gt, edges_pred, distance_metric, spacing),
            )  # type: ignore
        else:
            distances = (get_surface_distance(edges_pred, edges_gt, distance_metric, spacing),)  # type: ignore
            
        return convert_to_tensor(((edges_pred, edges_gt), distances, tuple(areas)), device=y_pred.device)  # type: ignore[no-any-return]

In [3]:
# calculation function
def calculation_all_metric(GT, PRED):
    metric = Metric(GT, PRED)
    metric.all_calculation()
    metric.print_metric()
    
    return metric.save_metric()

In [4]:
def remove_all_but_the_largest_connected_component(image: np.ndarray, for_which_classes: list, volume_per_voxel: float,
                                                   minimum_valid_object_size: dict = None):
    """
    removes all but the largest connected component, individually for each class
    :param image:
    :param for_which_classes: can be None. Should be list of int. Can also be something like [(1, 2), 2, 4].
    Here (1, 2) will be treated as a joint region, not individual classes (example LiTS here we can use (1, 2)
    to use all foreground classes together)
    :param minimum_valid_object_size: Only objects larger than minimum_valid_object_size will be removed. Keys in
    minimum_valid_object_size must match entries in for_which_classes
    :return:
    """
    if for_which_classes is None:
        for_which_classes = np.unique(image)
        for_which_classes = for_which_classes[for_which_classes > 0]

    assert 0 not in for_which_classes, "cannot remove background"
    largest_removed = {}
    kept_size = {}
    for c in for_which_classes:
        if isinstance(c, (list, tuple)):
            c = tuple(c)  # otherwise it cant be used as key in the dict
            mask = np.zeros_like(image, dtype=bool)
            for cl in c:
                mask[image == cl] = True
        else:
            mask = image == c
        # get labelmap and number of objects
        lmap, num_objects = ndimage.label(mask.astype(int))

        # collect object sizes
        object_sizes = {}
        for object_id in range(1, num_objects + 1):
            object_sizes[object_id] = (lmap == object_id).sum() * volume_per_voxel

        largest_removed[c] = None
        kept_size[c] = None

        if num_objects > 0:
            # we always keep the largest object. We could also consider removing the largest object if it is smaller
            # than minimum_valid_object_size in the future but we don't do that now.
            maximum_size = max(object_sizes.values())
            kept_size[c] = maximum_size

            for object_id in range(1, num_objects + 1):
                # we only remove objects that are not the largest
                if object_sizes[object_id] != maximum_size:
                    # we only remove objects that are smaller than minimum_valid_object_size
                    remove = True
                    if minimum_valid_object_size is not None:
                        remove = object_sizes[object_id] < minimum_valid_object_size[c]
                    if remove:
                        image[(lmap == object_id) & mask] = 0
                        if largest_removed[c] is None:
                            largest_removed[c] = object_sizes[object_id]
                        else:
                            largest_removed[c] = max(largest_removed[c], object_sizes[object_id])
    return image, largest_removed, kept_size

In [5]:
def post_processing(data, whole=False):
    
    
    processed, largest_removed, kept_size = remove_all_but_the_largest_connected_component(
        image=data,
        for_which_classes=[1],
        volume_per_voxel=3,
        minimum_valid_object_size=None  # 이 인자는 생략하거나 None으로 설정할 수 있dma
        )

    processed = ndimage.median_filter(processed, size=3)

    #klcc_transform=KeepLargestConnectedComponent(applied_labels=[1], is_onehot=None, independent=False, connectivity=True, num_components=1)
    
    #processed=klcc_transform(data)

    #erosioned = ndimage.binary_erosion(processed, iterations=3)
    
    #dilationed = ndimage.binary_dilation(erosioned, iterations=3)
    
    #processed = np.where(dilationed == 0,0,1)
    
    return processed

In [13]:
# define path (설정)
gt_folder = './DATASET_Synapse/unetr_pp_raw/unetr_pp_raw_data/Task02_Synapse/Task002_Synapse/seg_gt/test'
pred_folder = './output_synapse/3d_fullres/Task002_Synapse/unetr_pp_trainer_synapse__unetr_pp_Plansv2.1/fold_4/validation_raw/'
# 256x256

# make nii files
flag_post = True
pred_name = os.listdir(pred_folder)
pred_name = [nii for nii in pred_name if '.nii' in nii]
print(len(pred_name))

# 기타 변수 (rotation 안 되어 있다면 활용)
#tmp_lst=['volume-15.nii', 'volume-18.nii', 'volume-28.nii', 'volume-3.nii', 'volume-33.nii', 'volume-37.nii', 'volume-42.nii', 'volume-47.nii', 'volume-5.nii', 'volume-54.nii', 'volume-70.nii', 'volume-73.nii', 'volume-80.nii']

6


In [14]:
metric_df = pd.DataFrame(columns=['LiTS_id', 'DC', 'Jaccard', 'VOE', 'RAVD', 'ASSD', 'RMSD', 'MSSD', 'SEN', 'SPE'])

for i in tqdm(range(len(pred_name))):
    #resize = tio.Resize((256, 256, -1))
    pred = tio.ScalarImage(os.path.join(pred_folder, pred_name[i]))
    label = tio.ScalarImage(os.path.join(gt_folder, pred_name[i]+'.gz')) 

    ############ Synapse
    print("#### Synapse testing... ####")
    resample = tio.Resample() # currecntly, we check 1x1x1 voxel spacing cases.
    label.data = np.where(label.data >= 1, 1, 0)
    resampled = resample(label)
    label = resampled.data

    pred = np.array(pred)
    pred = pred[0]
    if flag_post: # post-processing
        pred = post_processing(pred)

    label = np.array(label)
    if len(np.unique(label)) != 2: # error check
        print(gt_path)
        raise Exception("No label")
    label = np.transpose(label[0], (2,1,0)) # TODO: 9/24 axial
    ############

    pred = pred[np.newaxis, np.newaxis, ]
    label = label[np.newaxis, np.newaxis, ]
    
    #print(pred.shape) # 5D (batch, class, axial, X, Y)
    #print(gt.shape) # 5D

    # calculation metric
    dc, jac, voe, ravd, assd, rmsd, mssd, sen, spe = calculation_all_metric(pred, label)

    # save
    metric_df.loc[i] = [pred_name[i], dc, jac, voe, ravd, assd, rmsd, mssd, sen, spe]

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]Call to deprecated function (or staticmethod) data. (Setting the image data with the property setter is deprecated. Use the set_data() method instead) -- Deprecated since version 0.18.16.


#### Synapse testing... ####
######### Start! ######### 


 17%|██████████████                                                                      | 1/6 [00:15<01:18, 15.65s/it]

######### Done! ######### 
DC score(upper): 0.9780687168067898
Jaccard score(upper): 0.9570787516658029
VOE score(lower): 0.04292124833419708
RAVD score(lower): 0.0057394264097986765
ASSD score(lower): 0.9701236486434937
RMSD score(lower): 3.1764180660247803
MSSD score(lower): 45.836666107177734
SEN score(upper): 0.9808917045593262
SPE score(upper): 0.999117374420166
#### Synapse testing... ####
######### Start! ######### 


 33%|████████████████████████████                                                        | 2/6 [00:28<00:54, 13.75s/it]

...

######### Done! ######### 
DC score(upper): 0.9751592012825298
Jaccard score(upper): 0.9515226194184363
VOE score(lower): 0.0484773805815637
RAVD score(lower): 0.013462689078330864
ASSD score(lower): 0.655688464641571
RMSD score(lower): 1.1047170162200928
MSSD score(lower): 8.366600036621094
SEN score(upper): 0.9686822891235352
SPE score(upper): 0.9993221759796143
#### Synapse testing... ####


Call to deprecated function (or staticmethod) data. (Setting the image data with the property setter is deprecated. Use the set_data() method instead) -- Deprecated since version 0.18.16.


######### Start! ######### 


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:03<00:00, 10.59s/it]

######### Done! ######### 
DC score(upper): 0.9670591195843194
Jaccard score(upper): 0.9362192337621019
VOE score(lower): 0.06378076623789808
RAVD score(lower): 0.02109188614028689
ASSD score(lower): 0.8601989150047302
RMSD score(lower): 1.6141307353973389
MSSD score(lower): 20.248456954956055
SEN score(upper): 0.9570712447166443
SPE score(upper): 0.9982506036758423


In [49]:
# show result
metric_df.mean(axis=0)

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


DC          0.975236
Jaccard     0.951824
VOE         0.048176
RAVD        0.014482
ASSD        0.770031
RMSD        1.560496
MSSD       18.110736
SEN         0.976592
SPE         0.998704
dtype: float64

In [50]:
metric_df

,LiTS_id,DC,Jaccard,VOE,RAVD,ASSD,RMSD,MSSD,SEN,SPE
0,hepaticvessel_001.nii,0.978069,0.957079,0.042921,0.005739,0.970124,3.176418,45.836666,0.980892,0.999117
1,hepaticvessel_005.nii,0.964816,0.932024,0.067976,0.001474,1.046064,2.268121,23.388031,0.965528,0.999244
2,hepaticvessel_009.nii,0.969595,0.940984,0.059016,0.015669,0.920815,2.152332,19.000000,0.962116,0.998897
3,hepaticvessel_010.nii,0.975617,0.952396,0.047604,0.027756,0.765583,1.250551,18.493242,0.962444,0.999632
4,hepaticvessel_011.nii,0.975159,0.951523,0.048477,0.013463,0.655688,1.104717,8.366600,0.968682,0.999322
5,hepaticvessel_013.nii,0.967059,0.936219,0.063781,0.021092,0.860199,1.614131,20.248457,0.957071,0.998251
6,hepaticvessel_015.nii,0.985463,0.971342,0.028658,0.002261,0.460188,0.870261,9.695360,0.986579,0.999219
7,hepaticvessel_027.nii,0.971904,0.945344,0.054656,0.041868,0.958401,1.695265,15.362291,0.952376,0.999651
8,hepaticvessel_043.nii,0.971460,0.944504,0.055496,0.024361,1.025684,2.870257,32.649654,0.983588,0.997989
9,hepaticvessel_044.nii,0.968588,0.939090,0.060910,0.007966,0.832036,1.597346,20.420578,0.972477,0.998867


In [51]:
# df save
metric_df.loc[-1] = metric_df.mean(axis=0)
metric_df.to_csv('./excel_result/MSD_post_test50.csv', index=True)

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
